In [10]:
import pandas as pd
import numpy as np
import optuna
import joblib
import warnings

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

warnings.filterwarnings("ignore", category=UserWarning)

In [11]:
print("📂 Loading dataset...")
df = pd.read_csv("cleaned_spam_dataset.csv")

X = df["Body"]
y = df["Label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"✅ Data split: {X_train.shape[0]} train, {X_test.shape[0]} test")

📂 Loading dataset...
✅ Data split: 22717 train, 5680 test


In [12]:
print("⚙️ Extracting TF-IDF features...")
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words="english"
)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
print(f"✅ TF-IDF shape: {X_train_tfidf.shape}")

⚙️ Extracting TF-IDF features...
✅ TF-IDF shape: (22717, 5000)


In [13]:
def objective(trial):
    model_name = trial.suggest_categorical("model", ["logreg", "rf", "xgb", "lgbm"])

    if model_name == "logreg":
        C = trial.suggest_float("C", 1e-3, 10.0, log=True)
        solver = trial.suggest_categorical("solver", ["liblinear", "lbfgs"])
        model = LogisticRegression(max_iter=1000, C=C, solver=solver, class_weight="balanced")

    elif model_name == "rf":
        n_estimators = trial.suggest_int("n_estimators", 100, 400)
        max_depth = trial.suggest_int("max_depth", 5, 30)
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            class_weight="balanced",
            random_state=42,
            n_jobs=-1
        )

    elif model_name == "xgb":
        eta = trial.suggest_float("eta", 0.01, 0.3)
        max_depth = trial.suggest_int("max_depth", 3, 10)
        subsample = trial.suggest_float("subsample", 0.6, 1.0)
        model = XGBClassifier(
            learning_rate=eta,
            max_depth=max_depth,
            subsample=subsample,
            n_estimators=200,
            colsample_bytree=0.8,
            eval_metric="logloss",
            use_label_encoder=False,
            random_state=42
        )

    elif model_name == "lgbm":
        num_leaves = trial.suggest_int("num_leaves", 31, 256)
        learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
        model = LGBMClassifier(
            num_leaves=num_leaves,
            learning_rate=learning_rate,
            n_estimators=200,
            random_state=42
        )

    # Cross-validation score (5-fold)
    scores = cross_val_score(model, X_train_tfidf, y_train, cv=5, scoring="f1_macro")
    return np.mean(scores)


In [15]:
print("\n🚀 Starting Optuna optimization (this may take a while)...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("\n🏁 OPTUNA COMPLETE!")
print("✅ Best Model Type:", study.best_trial.params["model"])
print("🏆 Best F1-Macro Score:", round(study.best_value, 4))
print("🔧 Best Params:", study.best_params)


[I 2025-11-08 17:44:59,245] A new study created in memory with name: no-name-1c057629-6fec-41fd-826a-d15b3802e31c



🚀 Starting Optuna optimization (this may take a while)...
[LightGBM] [Info] Number of positive: 5489, number of negative: 12684
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.234731 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 313573
[LightGBM] [Info] Number of data points in the train set: 18173, number of used features: 4874
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302041 -> initscore=-0.837595
[LightGBM] [Info] Start training from score -0.837595
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh

[I 2025-11-08 17:48:14,124] Trial 0 finished with value: 0.9609348841739876 and parameters: {'model': 'lgbm', 'num_leaves': 201, 'learning_rate': 0.21876618001169035}. Best is trial 0 with value: 0.9609348841739876.
[I 2025-11-08 17:52:56,177] Trial 1 finished with value: 0.9567199578528406 and parameters: {'model': 'xgb', 'eta': 0.254751245958846, 'max_depth': 8, 'subsample': 0.8768482020050549}. Best is trial 0 with value: 0.9609348841739876.
[I 2025-11-08 17:53:13,872] Trial 2 finished with value: 0.9311725985695407 and parameters: {'model': 'rf', 'n_estimators': 196, 'max_depth': 21}. Best is trial 0 with value: 0.9609348841739876.
[I 2025-11-08 17:53:14,396] Trial 3 finished with value: 0.9245878065713156 and parameters: {'model': 'logreg', 'C': 0.011857911384056308, 'solver': 'liblinear'}. Best is trial 0 with value: 0.9609348841739876.
[I 2025-11-08 17:53:15,036] Trial 4 finished with value: 0.9565120018698853 and parameters: {'model': 'logreg', 'C': 3.068015449862882, 'solver':

[LightGBM] [Info] Number of positive: 5489, number of negative: 12684
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.264585 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 313573
[LightGBM] [Info] Number of data points in the train set: 18173, number of used features: 4874
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302041 -> initscore=-0.837595
[LightGBM] [Info] Start training from score -0.837595
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

[I 2025-11-08 17:56:14,376] Trial 6 finished with value: 0.9599243484733208 and parameters: {'model': 'lgbm', 'num_leaves': 196, 'learning_rate': 0.21714624999587098}. Best is trial 0 with value: 0.9609348841739876.
[I 2025-11-08 17:56:14,802] Trial 7 finished with value: 0.9200068619135134 and parameters: {'model': 'logreg', 'C': 0.005209924728596343, 'solver': 'liblinear'}. Best is trial 0 with value: 0.9609348841739876.


...

[LightGBM] [Info] Number of positive: 5489, number of negative: 12684
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.227293 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 313573
[LightGBM] [Info] Number of data points in the train set: 18173, number of used features: 4874
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302041 -> initscore=-0.837595
[LightGBM] [Info] Start training from score -0.837595
[LightGBM] [Info] Number of positive: 5488, number of negative: 12685
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.224735 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 312866
[LightGBM] [Info] Number of data points in the train set: 18173, number of used features: 4883
[LightGBM] [

[I 2025-11-08 18:16:27,877] Trial 17 finished with value: 0.9642420979147437 and parameters: {'model': 'lgbm', 'num_leaves': 109, 'learning_rate': 0.08226740521077712}. Best is trial 17 with value: 0.9642420979147437.


[LightGBM] [Info] Number of positive: 5489, number of negative: 12684
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.225538 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 313573
[LightGBM] [Info] Number of data points in the train set: 18173, number of used features: 4874
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302041 -> initscore=-0.837595
[LightGBM] [Info] Start training from score -0.837595
[LightGBM] [Info] Number of positive: 5488, number of negative: 12685
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.225318 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 312866
[LightGBM] [Info] Number of data points in the train set: 18173, number of used features: 4883
[LightGBM] [

[I 2025-11-08 18:19:57,572] Trial 18 finished with value: 0.9636846344519757 and parameters: {'model': 'lgbm', 'num_leaves': 119, 'learning_rate': 0.10464939362461086}. Best is trial 17 with value: 0.9642420979147437.
[I 2025-11-08 18:20:04,925] Trial 19 finished with value: 0.9224117753429626 and parameters: {'model': 'rf', 'n_estimators': 109, 'max_depth': 16}. Best is trial 17 with value: 0.9642420979147437.



🏁 OPTUNA COMPLETE!
✅ Best Model Type: lgbm
🏆 Best F1-Macro Score: 0.9642
🔧 Best Params: {'model': 'lgbm', 'num_leaves': 109, 'learning_rate': 0.08226740521077712}


In [16]:
best_params = study.best_params
model_type = best_params.pop("model")

if model_type == "logreg":
    model = LogisticRegression(max_iter=1000, **best_params)
elif model_type == "rf":
    model = RandomForestClassifier(random_state=42, n_jobs=-1, **best_params)
elif model_type == "xgb":
    model = XGBClassifier(eval_metric="logloss", use_label_encoder=False, random_state=42, **best_params)
elif model_type == "lgbm":
    model = LGBMClassifier(random_state=42, **best_params)

model.fit(X_train_tfidf, y_train)
print(f"\n✅ Retrained best model ({model_type}) on full training data.")

[LightGBM] [Info] Number of positive: 6861, number of negative: 15856
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.286774 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 371904
[LightGBM] [Info] Number of data points in the train set: 22717, number of used features: 4918
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302021 -> initscore=-0.837695
[LightGBM] [Info] Start training from score -0.837695

✅ Retrained best model (lgbm) on full training data.


In [ ]:
joblib.dump(model, "model/best_spam_model.pkl")
joblib.dump(tfidf, "model/tfidf_vectorizer.pkl")

print(f"💾 Saved best model and TF-IDF vectorizer.")
print("🎯 Best Model Type:", model_type)
print("🎯 Best Hyperparameters:", best_params)
print("✨ Optuna tuning complete — ready for evaluation or deployment.")

💾 Saved best model and TF-IDF vectorizer.
🎯 Best Model Type: lgbm
🎯 Best Hyperparameters: {'num_leaves': 109, 'learning_rate': 0.08226740521077712}
✨ Optuna tuning complete — ready for evaluation or deployment.
